In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import yaml


In [ ]:
#Read main path
with open('../path_main.txt', 'r') as file:    path_main = file.read()

dir_data_HSIchg = f'{path_main}Data/Plot_preparation/HSI_changes/'
dir_data_ThrExc = f'{path_main}Data/Plot_preparation/Threshold_Exceedance/'
dir_data_HWMId  = f'{path_main}Data/Plot_preparation/HWMId/'
dir_scripts     = f'{path_main}Scripts/'
dir_names       = f'{path_main}Scripts/Model_lists/'
dir_fig         = f'{path_main}Figures/Paper_v2/'
if not os.path.exists(dir_fig): os.mkdir(dir_fig)
    

## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade',
          'Zagreb', 'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam',
          'London', 'Dublin', 'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk','Vilnius', 
          'Riga', 'Moscow', 'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

cities = np.array(cities)

#Define HSIs
HSIs = ['TX', 'TN']
    
HSI_out = {'TN': 'TN', 
           'TX': 'TX'}

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)
    
#Define scenarios and variables
RCP = 'rcp85'

#Define models and RCPs which should be used
all_models = dict()
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))

#Define warming levels
EMT_change = '3.0K'#'1.0K', '2.0K', 


## Plot preparation

In [ ]:
#Titles 1
tit_chg = dict()
tit_chg['JJA_mean'] = ' (JJA mean)'
tit_chg['Q90']      = ' (JJA, Q90)'
tit_chg['Ymax']     = 'x change'

#Titles 2
titles = lambda HSI, THR_level: 'Exceedance ' + HSI + '>' + THR_levels_vals[HSI + '_' + THR_level] + '°C'

#Units
units = 'd/y'

#Labels
texts = ['a)', 'b)', 'c)', 'd)']

#Color, scaling, and alpha
edgecolor = 'k'
sc_scale  = 200
sc_offset = 400
alpha     = 0.8

#Define ranking vector
ranks = np.arange(1, len(cities) + 1)

THR_levels = dict()
THR_levels['TN']= [1, 2, 3, 4]
THR_levels['TX']= [1, 2, 3, 4]

THR_levels_vals = dict()
THR_levels_vals['TN_Level1'] = '15'
THR_levels_vals['TN_Level2'] = '17'
THR_levels_vals['TN_Level3'] = '20'
THR_levels_vals['TN_Level4'] = '23'
THR_levels_vals['TX_Level1'] = '25'
THR_levels_vals['TX_Level2'] = '27'
THR_levels_vals['TX_Level3'] = '30'
THR_levels_vals['TX_Level4'] = '33'

vmax_all = dict()
vmax_all['TN']      = 120
vmax_all['TX']      = 120

#Define colormap
cmap   = matplotlib.cm.get_cmap('Reds', len(cities))
colors = cmap(np.linspace(0, 0.9,  len(cities)))


## Plot for EMT

In [ ]:
# out_name = '_3x3'
out_name = ''

#Select which time method to use for HSI changes ('JJA_mean', 'Q90', 'Ymax')
t_method = 'Ymax'

transformations = ['no_trans']#, 'trans_mean', 'trans_z']

#Loop over HSIs
for i6, HSI in enumerate(HSIs):

    #Loop over transformations
    for transformation in transformations:

        #Create figure
        fig, axes = plt.subplots(2, 2, figsize=(15, 11), subplot_kw=dict(projection=ccrs.Robinson()))
        axes = axes.flatten()
        plt.subplots_adjust(hspace=0.4, wspace=0.2)


        #Loop over levels
        for i5, level in enumerate(THR_levels[HSI]):

            ax = axes[i5]

            #Add coastlines and borders
            ax.coastlines(resolution='50m', linewidth=0.75, color='#737373', zorder=0)
            ax.add_feature(cfeature.LAND, facecolor='#f0f0f0')
            ax.add_feature(cfeature.OCEAN, facecolor='aliceblue')
            ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='#969696', zorder=0)
            ax.set_extent([-12, 45, 35, 62], crs=ccrs.PlateCarree())
            ax.spines['geo'].set_zorder(30)


            #Read data for threshold exceedance
            THR_level = 'Level' + str(level)
            data_ThrExc = xr.open_dataset(dir_data_ThrExc + 'EURO-CORDEX' + out_name + '/HSIs-ThresholdExceedance_rcp85_EMT.nc')
            data_ThrExc = data_ThrExc.sel(city=cities, EMT_change=EMT_change, THR_level=THR_level, transformation=transformation)[HSI]
            data_ThrExc_val = data_ThrExc.values

            #Save data in dict for plotting
            data_plot = data_ThrExc

            #Calculate ranking average across models
            data_ThrExc_val = np.median(data_ThrExc_val, axis=0)

            #Sort values
            data_ThrExc_val = np.argsort(data_ThrExc_val)

            #Create empty arrays
            ranking_ThrExc = np.empty(data_ThrExc_val.shape) * np.NaN

            #Loop over models and get ranking of each city
            ranking_ThrExc[data_ThrExc_val] = ranks

            #Order cities according to average ranking
            cities_ordered = np.flip(cities[np.argsort(ranking_ThrExc)])

            #Loop over cities
            x = [];  y = []
            for i0, city in enumerate(cities_ordered):

                #Get coordinates
                coord = city_coords[city]

                #Write number
                if i0<5:
                    ax.text(coord[1], coord[0], str(i0+1), transform=ccrs.PlateCarree(),
                            color='w', fontsize=10, fontweight='bold', ha='center', va='center')
                elif i0>len(cities_ordered) - 6:
                    ax.text(coord[1], coord[0], str(i0+1), transform=ccrs.PlateCarree(),
                            color='k', fontsize=10, ha='center', va='center')

                x.append(coord[1])
                y.append(coord[0])

            #Get multi-model median
            val = data_plot.median('model').sel(city=cities_ordered)


            vmin = 0
            vmax = vmax_all[HSI]

            #Scatter plot
            sc = ax.scatter(x, y, s=sc_scale*val/vmax + sc_offset, transform=ccrs.PlateCarree(), c=val, edgecolor=edgecolor,
                            cmap=cmap, vmax=vmax, vmin=vmin, alpha=alpha)

            val_h = np.arange(0, vmax+1, 30)
            x_h = np.ones(len(val_h))
            y_h = np.ones(len(val_h))    
            for x, y, v in zip(x_h, y_h, val_h):

                sc_h = ax.scatter(x, y, s=sc_scale*v/vmax + sc_offset, transform=ccrs.PlateCarree(), color=cmap(v/vmax),
                                  edgecolor=edgecolor, label=v, alpha=alpha)

            #Number of legend entries
            N_leg = len(val_h)

             #Set title
            ax.set_title(titles(HSI, THR_level), fontsize=20, fontweight='bold', pad=20)

            #Prepare legend
            if N_leg>5:
                ncol = int(len(ranges[HSI + '_' + EMT_change])/2)
                y_bbox = -0.38
                y_txt  = -0.53
            else:
                ncol = N_leg
                y_bbox = -0.23
                y_txt  = -0.33

            columnspacing = 1.2
            handletextpad = 0.3


            #Legend
            legend = ax.legend(fontsize=17, bbox_to_anchor=(0.5, y_bbox), frameon=False, loc='lower center',
                               labelspacing=1, columnspacing=columnspacing, handletextpad=handletextpad, ncol=ncol)
            ax.text(0.5, y_txt, units, fontsize=24, va='bottom', ha='center', transform=ax.transAxes)

            #Write letters
            ax.text(0.03, 0.96, texts[i5], fontsize=16, fontweight='bold', va='top', ha='left', transform=ax.transAxes)
            
        #Save figure
        if HSI=='TX' and transformation=='no_trans':  fig_name = 'FigS3'
        else:                                         fig_name = 'FigS'
        plt.savefig(dir_fig + fig_name + '_HeatStressMap-AllLevels_' + HSI + '_' + transformation + '_dEMT' + EMT_change + '_EURO-CORDEX' + out_name + '.png',
                    bbox_inches='tight', dpi=300)
